## Exercise 03. Aggregations ##

## Imports ##

In [22]:
import pandas as pd
import sqlite3

## Task ##

## Test schema ##
1. Создайте подключение к базе данных с использованием библиотеки `sqlite3`.
2. Получите схему таблицы `test`.

In [23]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
schema = pd.read_sql('PRAGMA table_info(test);', conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Test 10 ##
3. Получите только первые десять строк таблицы `test`, чтобы посмотреть, как она выглядит.

In [24]:
first_ten = pd.read_sql('SELECT * FROM test LIMIT 10;', conn)
first_ten.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Minimum delta ##
4. Найдите минимальное значение дельты между первым коммитом и сроком выполнения соответствующей лабораторной работы для всех пользователей, используя только один запрос.
   - Сделайте это путём объединения таблицы с таблицей `deadlines`.
   - Разница должна отображаться в часах.
   - Не учитывайте лабораторную работу `project1`, так как у неё более длительные сроки и она будет выбросом.
   - Значение должно быть сохранено в датафрейме `df_min` вместе с соответствующим `uid`.

In [25]:
deadlines = pd.read_sql('SELECT * FROM deadlines', conn)
deadlines

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [26]:
df_min = pd.read_sql('SELECT uid, MIN((deadlines - strftime("%s", first_commit_ts)) / 3600.0) AS delta_hours FROM test JOIN deadlines ON test.labname = deadlines.labs WHERE labname != "project1"', conn)
df_min

,uid,delta_hours
0,user_25,2.8675


## Maximum delta ##
5. Сделайте то же самое для максимального значения, используя только один запрос. Название датафрейма — `df_max`.

In [27]:
df_max = pd.read_sql('SELECT uid, MAX((deadlines - strftime("%s", first_commit_ts)) / 3600.0) AS delta_hours FROM test JOIN deadlines ON test.labname = deadlines.labs WHERE labname != "project1"', conn)
df_max

,uid,delta_hours
0,user_30,202.385


## Medium delta ##
6. Аналогично для среднего значения. Используйте только один запрос. В этот раз, ваш датафрейм не должен содержать колонку `uid`. Название датафрейма — `df_avg`.


In [28]:
df_avg = pd.read_sql('SELECT AVG((deadlines - strftime("%s", first_commit_ts)) / 3600.0) AS delta_hours FROM test JOIN deadlines ON test.labname = deadlines.labs WHERE labname != "project1"', conn)
df_avg

,delta_hours
0,89.687841


## Check hypothesis ##
7. Нам нужно проверить гипотезу, что пользователи, посетившие новостную ленту всего несколько раз, имеют меньшую дельту между первым коммитом и сроком сдачи. Для этого вычислите коэффициент корреляции между количеством просмотров страниц и разницей.
   - Используя только один запрос, создайте таблицу с колонками: "uid", "avg_diff" и "pageviews".
   - "uid" — это идентификаторы пользователей, существующие в `test`.
   - "avg_diff" — это средняя дельта между первым коммитом и сроком выполнения лабораторной работы для каждого пользователя.
   - "pageviews" — это количество посещений новостной ленты для каждого пользователя.
   - Не учитывайте лабораторную работу `project1`.
   - Сохраните результат в датафрейм `views_diff`.
   - Используйте метод Pandas `corr()` для вычисления коэффициента корреляции между количеством просмотров и разницей.
8. Закройте соединение.

In [29]:
query = '''
SELECT 
    t.uid,
    AVG((t.deadlines - strftime("%s", t.first_commit_ts)) / 3600.0) AS avg_diff,
    COUNT(p.uid) AS pageview_count
FROM 
    (SELECT uid, deadlines, first_commit_ts
     FROM test
     JOIN deadlines ON test.labname = deadlines.labs
     WHERE labname != "project1") AS t
LEFT JOIN pageviews p ON t.uid = p.uid
GROUP BY t.uid
'''

views_diff = pd.read_sql(query, conn)
views_diff

,uid,avg_diff,pageview_count
0,user_1,65.119778,140
1,user_10,75.242444,445
2,user_14,159.568796,429
3,user_17,62.207667,235
4,user_18,6.368148,9
5,user_19,99.440417,64
6,user_21,96.111181,40
7,user_25,93.474944,895
8,user_28,86.793833,745
9,user_3,105.738222,1585


In [30]:
views_diff.corr(numeric_only=True)

,avg_diff,pageview_count
avg_diff,1.000000,0.185042
pageview_count,0.185042,1.000000


## Вывод: корреляция невысокая ##
Коэффициент корреляции до 0.3 в принципе можно принять к сведению, но не рассматривать
### Чувство соперничества не победило лень ###

In [31]:
conn.close()